In [1]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join("..")))

In [2]:
import ee
import geemap
from utils import ee_utils, date_utils, plot_config
from utils.ee_utils import back_to_float

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from typing import Dict, List, Tuple, Union

import warnings
from validation.irrigation_validation import IrrigationValidator
from validation.plotting import ValidationPlotter

from pathlib import Path 

warnings.filterwarnings("ignore")

In [3]:
from filter_nutzungsflaechen import (
    get_crops_to_exclude,
    get_rainfed_reference_crops,
    create_crop_filters,
    filter_crops,
    add_double_cropping_info,
)

---

In [4]:
PATH_TO_AOI = "projects/thurgau-irrigation/assets/Thurgau/thrugau_borders_2024"

aoi_feature_collection = ee.FeatureCollection(PATH_TO_AOI)
aoi_geometry = aoi_feature_collection.geometry().simplify(500)

AOI = aoi_geometry.buffer(100)

In [5]:
# # Load field collections
# fields_w_estimates = ee.FeatureCollection(
#     "projects/thurgau-irrigation/assets/ZH_Nutzungsflaechen_2/2022_with_irrigation_estimates"
# )
modelled_fields = ee.FeatureCollection(
    "projects/thurgau-irrigation/assets/Thurgau/ET_blue_m3_ha_yr_per_field_from_monthly_Landsat_10m_2018-2022/ET_blue_m3_ha_yr_per_field_from_monthly_landsat_10m_TG_2022"
)

# Print initial count
# print(f"Initial fields count: {fields_w_estimates.size().getInfo()}")

# Get crops to exclude
crops_to_exclude = get_crops_to_exclude().union(get_rainfed_reference_crops())


# Function to set estimated_irrigated_volume to 0 for excluded crop types
def set_volume_for_excluded_crops(feature):
    is_excluded = ee.List(list(crops_to_exclude)).contains(feature.get("NUTZUNG"))
    return feature.set(
        "estimated_irrigated_volume",
        ee.Number(
            ee.Algorithms.If(
                is_excluded,
                0,  # Set to 0 if crop type is excluded
                feature.get("estimated_irrigated_volume"),  # Keep original value
            )
        ),
    )


# Apply the function to all fields
# fields_w_estimates = fields_w_estimates.map(set_volume_for_excluded_crops)

# print(f"Total fields with estimates: {fields_w_estimates.size().getInfo()}")

# # Get all parcels and join with modelled fields
# valid_parcels = fields_w_estimates.aggregate_array("PARZNR").distinct()
# filtered_modelled_fields = modelled_fields.filter(
#     ee.Filter.inList("PARZNR", valid_parcels)
# )

print(f"Total modelled fields: {modelled_fields.size().getInfo()}")

Total modelled fields: 58299


In [8]:
IRRIGATION_EFFICIENCY = 0.5

palette = ["white", "red", "orange", "yellow", "green"]

# Define visualization parameters for the images
visualization = {
    "min": 100,
    "max": 1000,
    "palette": palette,
}

# Define visualization parameters for the feature collections - borders only
fields_style = {
    "color": "black",
    "width": 1,
    "fillcolor": None,
}


# # Convert fields_w_estimates to an image for gradient visualization
# fields_w_estimates_image = fields_w_estimates.reduceToImage(
#     properties=["estimated_irrigated_volume"], reducer=ee.Reducer.first()
# )

filtered_modelled_fields_image = modelled_fields.reduceToImage(
    properties=["ET_blue_m3_ha_yr"], reducer=ee.Reducer.first()
)

# feature_to_check = ee.FeatureCollection(
#     filtered_modelled_fields.filter(ee.Filter.eq("PARZNR", "UH3669"))
# )

# Divide by irrigation efficiency to get the actual irrigation volume
filtered_modelled_fields_image = filtered_modelled_fields_image.divide(
    IRRIGATION_EFFICIENCY
)

# Create a map centered on the area of interest
Map = geemap.Map()
Map.center_object(AOI, zoom=12)

# Add the gradient visualization layers to the map
# Map.addLayer(fields_w_estimates_image, visualization, "Irrigation Volume (Gradient)")
Map.addLayer(filtered_modelled_fields_image, visualization, "Modelled ET_blue_m3_ha_yr")
# Map.addLayer(fields_w_estimates, fields_style, "Fields Borders")

Map.addLayer(AOI, {}, "AOI")

Map.add_colorbar(
    visualization, label="Irrigation Volume (m³/ha)", orientation="horizontal"
)


Map

Map(center=[47.56858787382066, 9.092720596553875], controls=(WidgetControl(options=['position', 'transparent_b…

In [9]:
# Compute the area of all fields in filtered_modelled_fields where the ET_blue_m3_ha_yr is greater than 0
total_irrigated_area = (
    modelled_fields.filter(ee.Filter.gt("ET_blue_m3_ha_yr", 100))
    .geometry()
    .area()
    .divide(10000)
    .getInfo()
)
print(f"Total irrigated area: {total_irrigated_area:.2f} ha")

# Print how many hectares are not irrigated
total_non_irrigated_area = (
    modelled_fields.filter(ee.Filter.lte("ET_blue_m3_ha_yr", 100))
    .geometry()
    .area()
    .divide(10000)
    .getInfo()
)
print(f"Total non-irrigated area: {total_non_irrigated_area:.2f} ha")

# # Print area of fields with estimated irrigation volume
# total_estimated_irrigated_area = (
#     fields_w_estimates.filter(ee.Filter.gt("estimated_irrigated_volume", 0))
#     .geometry()
#     .area()
#     .divide(10000)
#     .getInfo()
# )
# print(f"Total estimated irrigated area: {total_estimated_irrigated_area:.2f} ha")

# # Print area of estimated non-irrigated fields
# total_estimated_non_irrigated_area = (
#     fields_w_estimates.filter(ee.Filter.eq("estimated_irrigated_volume", 0))
#     .geometry()
#     .area()
#     .divide(10000)
#     .getInfo()
# )
# print(f"Total estimated non-irrigated area: {total_estimated_non_irrigated_area:.2f} ha")

Total irrigated area: 11655.08 ha
Total non-irrigated area: 40674.03 ha
